## Chapter 5 - Digital Building Blocks 



### 5.1 Introduction

### 5.2 Arithmetic Circuit 

#### 5.2.1 Summation

3 type of adders are covered: 
1. Ripple-carry adder: A group of full adders are connected in cascade 
2. Carry-lookahead adder: 
3. Prefix adder: 

#### 5.2.2 Subtraction


### 5.3 Number Systems 

### 5.4 Sequential Building Blocks 

### 5.5 Memory Array 

### 5.6 Logic Array 

### 5.7 Summary 


## Exercises:

#### Exercise 5.1 

a) Ripple-Carry adder: <br>
$t_{ripple} = (450ns)64 = 28.8\mu s$ <br>

(b) Carry-Lookahead adder: <br>
$t_{CLA} = (150ns) + 6(150ns) + (64/4-1)2(150ns) + 4(450ns) = 7.35\mu s$

(c) Prefix Adder: <br>
$t_{PA} = (100ns) + (200ns)\log_2(64) + (100ns) = 1.4\mu s$


#### Exercise 5.2

(a)
1. <b>Ripple-Carry adder: </b> <br> 
<u>Area</u>: Each full adder requires 5 two-input gates to build each of the 64 full adders. Thus, the full area should be $(5)(64)15\mu m^2 = 4800\mu m^2$<br>
<u>Delay</u>: There are 3 two-input gates along the critical path of a full adder. Thus, the full propagation delay should be: $(3)(50ps)(64) = 9.6 \mu s$. <br>
<u>Power</u>: All 2-input gates are equivalent to capacitors in parallel as they are supplied voltage in parallel. Additionally, we can assume the probability of bit switching ($ar$) is 1 to obtain the worst case. Thus, dynamics power should be $P_{dyn} = arf_{C}C_{L}V_{DD}^2 = (1)(100\cdot10^6Hz)(20\cdot10^{-15}F)(320 \text{ gates})(1.2V)^2 = 921.6mW$ <br><br>

2. <b>Carry-Lookahead adder: </b> <br> 
<u>Area</u>:  5 two-input gates are needed to build each of the 16 CLA blocks' 4 full adders. Also, each CLA block uses 19 additional two-input gates to compute $P_i$, $G_i$ and $C_{out}$. Thus, the full area should be $16((4)5 + 19)(15\mu m^2) = (624)15\mu m^2 = 9360 \mu m^2$ <br>
<u>Delay</u>: two-input gates and full adders have a delay of 50ps and 150ps respectively. Thus, $t_{CLA} = (50ns) + 6(50ns) + (64/4-1)2(50ns) + 4(150ns) = 2.45\mu s$ <br>
<u>Power</u>: Assuming again the probability of bit switching ($ar$) is 1 to obtain the worst case, dynamics power should be $P_{dyn} = arf_{C}C_{L}V_{DD}^2 = (1)(100\cdot10^6Hz)(20\cdot10^{-15}F)(624 \text{ gates})(1.2V)^2 = 1509.12 mW$

(b) 
A Ripple-Carry adders require roughly half the space and half the power of Carry-Lookahead adders. However, it's nearly 4 times slower. 


Formula for computing dynamic power found here: [https://www.iue.tuwien.ac.at/phd/schrom/node12.html]

#### Exercise 5.3

As discussed in question 5.2, Ripple-Carry adders have the advantage of using approximately two times less two-input gates than carry-lookahead adders. This translates to needing less space and using less power. Nevertheless, its propagation delay is significantly higher.  

#### Exercise 5.4
```
module testbench_4_39();

  logic[15:0] A,B,S;
  logic C_in; 
  
  prefix_sum dut(A,B,C_in, S); 

  initial 
    begin

      A = 16'b0; B = 16'b1; C_in = 0; #2;
	  A = 8632; B = 28033; C_in = 0; #3;
      
    end


  //  Optional: save signals to display them later on EDAplayground's EPWave 
    initial begin
      $dumpfile("dump.vcd");
      $dumpvars(1);
    end
endmodule
```

```
module prefix_sum(input  logic[15:0] A, B, 
                  input  logic C_in, 
                  output logic[15:0] S); 

  // precomputation; 
  logic[15:0] P, G; 
  assign G = A&B; 
  assign P = A|B;

  
  logic G_14_13, G_12_11, G_10_9, G_8_7, G_6_5, G_4_3, G_2_1, G_0_n1;
  assign G_14_13 = G[14] | (&{P[14]}& G[13]); 
  assign G_12_11 = G[12] | (&{P[12]}& G[11]); 
  assign G_10_9  = G[10] | (&{P[10]}& G[9]); 
  assign G_8_7   = G[8]  | (&{P[8]} & G[7]); 
  assign G_6_5   = G[6]  | (&{P[6]} & G[5]); 
  assign G_4_3   = G[4]  | (&{P[4]} & G[3]); 
  assign G_2_1   = G[2]  | (&{P[2]} & G[1]); 
  assign G_0_n1  = G[0]  | (&{P[0]} & C_in); 
  
  
  logic G_14_11, G_13_11, G_10_7, G_9_7, G_6_3, G_5_3, G_2_n1, G_1_n1; 
  assign G_14_11 = G_14_13 | (&{P[14:13]} & G_12_11); 
  assign G_13_11 = G[13]   | (&{P[13]}    & G_12_11); 
  assign G_10_7  = G_10_9  | (&{P[10:9]}  & G_8_7); 
  assign G_9_7   = G[9]    | (&{P[9]}    & G_8_7); 
  assign G_6_3   = G_6_5   | (&{P[6:5]}   & G_4_3); 
  assign G_5_3   = G[5]    | (&{P[5]}     & G_4_3); 
  assign G_2_n1  = G_2_1   | (&{P[2:1]}   & G_0_n1); 
  assign G_1_n1  = G[1]    | (&{P[1]}     & G_0_n1); 

  
  logic G_14_7, G_13_7, G_12_7, G_11_7, G_6_n1, G_5_n1, G_4_n1, G_3_n1; 
  assign G_14_7 = G_14_11 | (&{P[14:11]} & G_10_7); 
  assign G_13_7 = G_13_11 | (&{P[13:11]} & G_10_7); 
  assign G_12_7 = G_12_11 | (&{P[12:11]} & G_10_7); 
  assign G_11_7 = G[11]   | (&{P[11]}    & G_10_7); 
  assign G_6_n1 = G_6_3   | (&{P[6:3]}   & G_2_n1); 
  assign G_5_n1 = G_5_3   | (&{P[5:3]}   & G_2_n1); 
  assign G_4_n1 = G_4_3   | (&{P[4:3]}   & G_2_n1); 
  assign G_3_n1 = G[3]    | (&{P[3]}     & G_2_n1); 

  logic G_14_n1, G_13_n1, G_12_n1, G_11_n1, G_10_n1, G_9_n1, G_8_n1, G_7_n1; 
  assign G_14_n1 = G_14_7 | (&{P[14:7]} & G_6_n1); 
  assign G_13_n1 = G_13_7 | (&{P[13:7]} & G_6_n1); 
  assign G_12_n1 = G_12_7 | (&{P[12:7]} & G_6_n1); 
  assign G_11_n1 = G_11_7 | (&{P[11:7]} & G_6_n1); 
  assign G_10_n1 = G_10_7 | (&{P[10:7]} & G_6_n1); 
  assign G_9_n1  = G_9_7  | (&{P[9:7]}  & G_6_n1); 
  assign G_8_n1  = G_8_7  | (&{P[8:7]}  & G_6_n1); 
  assign G_7_n1  = G[7]   | (&{P[7]}    & G_6_n1); 
  
  //logic G_n_n1 = {G_14_n1, G_13_n1, G_12_n1, G_11_n1, G_10_n1, G_9_n1, G_8_n1, G_7_n1, G_6_n1, G_5_n1, G_4_n1, G_3_n1, G_2_n1, G__n1, }
  
  assign S[15] = G_14_n1 ^ (A[15]^B[15]);
  assign S[14] = G_13_n1 ^ (A[14]^B[14]);
  assign S[13] = G_12_n1 ^ (A[13]^B[13]);
  assign S[12] = G_11_n1 ^ (A[12]^B[12]);
  
  assign S[11] = G_10_n1 ^ (A[11]^B[11]);
  assign S[10] = G_9_n1  ^ (A[10]^B[10]);
  assign S[9]  = G_8_n1  ^ (A[9] ^B[9]);
  assign S[8]  = G_7_n1  ^ (A[8] ^B[8]);
  assign S[7]  = G_6_n1  ^ (A[7] ^B[7]);
  assign S[6]  = G_5_n1  ^ (A[6] ^B[6]);
  assign S[5]  = G_4_n1  ^ (A[5] ^B[5]);
  assign S[4]  = G_3_n1  ^ (A[4] ^B[4]);
  assign S[3]  = G_2_n1  ^ (A[3] ^B[3]);
  assign S[2]  = G_1_n1  ^ (A[2] ^B[2]);
  assign S[1]  = G_0_n1  ^ (A[1] ^B[1]);
  assign S[0]  = A[0] ^B[0];

endmodule 
```

#### Exercise 5.5


![alt text](images\P5_5.PNG "Title")

#### Exercise 5.6





#### Exercise 5.7

(a) <br>
A priority circuit with a delay that increases in logarithmic scale can be achieve in 3 steps. Let $A_i$ for $i=\{0,N-1\}$ be the circuit inputs. 

1. Step 1: Compute values of $S_i$ for $i = 0$ to $N-3$ which are outputs of summations that follow a binomial tree structure. This part of the circuit has $N-3$ two-input AND gates and has a delay of $(\log2(N)-1)t_{pd,AND}$. 
2. Step 2:  Compute the values of $B_i$ for $i = 0$ to $N-1$. $B$ represent the position of the most significant 1. In other word,$B_i = 1$ iff $A_i = 1$ and $A_j = 0$ for $j > i$. This step has a delay of $(\log2(N)-1)(t_{pd,AND} +t_{pd,NOT})$. This step can be viewed as a bisection search for the value of $i$ such that $B_i = 1$
3. Step 3: Step 3 consists of computing $Y_i$ for $i = 0$ to $\log2(N)-1$. This has a delays of $(\log2(N)-1)(t_{pd,AND})$. 

The total delay for this circuit is $(\log2(N)-1)(3t_{pd,AND} + t_{pd,NOT})$

Below is a draft of the circuit when 16 inputs are present. <br> 
![alt text](images\P5_7.PNG "Title")

(b) 


<b> Verilog Testbench </b> 
```
module testbench_4_39();

  logic[15:0] A;
  logic[3:0] Y;

  
  log2_priority dut(A,Y); 

  initial 
    begin

      A = 16'b00001010; #2; // 3
	  A = 16'b10100010; #3; // 7
      A = 16'b00011111; #3; // 4
      A = 16'b00000000; #3; // 0 
      A = 16'b00000001; #3; // 1
    end


  //  Optional: save signals to display them later on EDAplayground's EPWave 
    initial begin
      $dumpfile("dump.vcd");
      $dumpvars(1);
    end
endmodule
```


<b> Verilog Code </b> 
```
// Code your design here
module log2_priority(input logic[15:0] A, 
                     output logic[3:0] Y); 
  
  // step 1
  logic[13:0] S; 
  
  assign S[0] = |{A[1:0]}; 
  assign S[1] = |{A[3:2]}; 
  assign S[2] = |{A[5:4]}; 
  assign S[3] = |{A[7:6]}; 
  assign S[4] = |{A[9:8]}; 
  assign S[5] = |{A[11:10]}; 
  assign S[6] = |{A[13:12]}; 
  assign S[7] = |{A[15:14]}; 
  assign S[8] = |{S[1:0]};
  assign S[9] = |{S[3:2]};
  assign S[10] = |{S[5:4]};
  assign S[11] = |{S[7:6]};
  assign S[12] = |{S[9:8]};
  assign S[13] = |{S[11:10]};  
    
    
  // step 2
  logic[15:0] B;
  
  assign B[15] = A[15]; 
  assign B[14] = A[14]&(~A[15]);
  assign B[13] = A[13]&(~S[7]); 
  assign B[12] = A[12]&(~S[7])&(~A[13]); 
  assign B[11] = A[11]&(~S[11]); 
  assign B[10] = A[10]&(~S[11])&(~A[11]); 
  assign B[9] = A[9]&(~S[11])&(~S[5]); 
  assign B[8] = A[8]&(~S[11])&(~S[5])&(~A[9]);  
  assign B[7] = A[7]&(~S[13]); 
  assign B[6] = A[6]&(~S[13])&(~A[7]); 
  assign B[5] = A[5]&(~S[13])&(~S[3]); 
  assign B[4] = A[4]&(~S[13])&(~S[3])&(~A[5]);  
  assign B[3] = A[3]&(~S[13])&(~S[9]); 
  assign B[2] = A[2]&(~S[13])&(~S[9])&(~A[3]);
  assign B[1] = A[1]&(~S[13])&(~S[9])&(~S[1]);
  assign B[0] = A[0]&(~S[13])&(~S[9])&(~S[1])&(~A[0]);  

  // step 3
  assign Y[3] = |{B[15:8]}; 
  assign Y[2] = |{B[15:12]} | |{B[7:4]}; 
  assign Y[1] = B[15]|B[14]|B[11]|B[10]|B[7]|B[6]|B[3]|B[2];  
  assign Y[0] = B[15]|B[13]|B[11]|B[9]|B[7]|B[5]|B[3]|B[1];
  
endmodule 
```

#### Exercise 5.8

#### Exercise 5.9


```
module testbench_4_29();

  logic[31:0] A, B, Y;
  logic[2:0] F; 
  logic C_out; 

  ALU_32bits dut(A, B, F, Y, C_out); 

  initial 
    begin
      
       // additions...
      A = 0; B = 0; F = 3'b010; #1;
      A = 0; B = 1; F = 3'b010; #1; 
      A = 0; B = 2; F = 3'b010; #1;
      A = 0; B = 3; F = 3'b010; #1;
      
      // subtractions...
      A = 0; B = 0; F = 3'b110; #1; 
      A = 0; B = 1; F = 3'b110; #1; 
      A = 0; B = 2; F = 3'b110; #1;
      A = 0; B = 3; F = 3'b110; #1; 
      
      // ANDs
      A = 32'b000111; B = 32'b000111; F = 3'b000; #1; // 7
      A = 32'b101010; B = 32'b010101; F = 3'b000; #1; // 0 
      A = 32'b111111; B = 32'b100111; F = 3'b000; #1; // 39
      A = 32'b111000; B = 32'b110111; F = 3'b000; #1; // 48
      
      // ORs
      A = 32'b000111; B = 32'b000111; F = 3'b001; #1; // 7
      A = 32'b101010; B = 32'b010101; F = 3'b001; #1; // 0x3F
      A = 32'b111111; B = 32'b100111; F = 3'b001; #1; // 0x3F
      A = 32'b111000; B = 32'b110111; F = 3'b001; #1; // 0x3F
      
      // SLT
      A = 32'b000011; B = 32'b000111; F = 3'b111; #1; // 1
      A = 32'b101010; B = 32'b010101; F = 3'b111; #1; // 0
      A = 32'b111111; B = 32'b100111; F = 3'b111; #1; // 0
      A = 32'b001010; B = 32'b010101; F = 3'b111; #1; // 1
      A = 32'b001010; B = 32'b010111; F = 3'b111; #1; // 1
      A = 32'b101010; B = 32'b010101; F = 3'b111; #1; // 0
      A = 32'b111111; B = 32'b100111; F = 3'b111; #1; // 0

    end

  //  Optional: save signals to display them later on EDAplayground's EPWave 
    initial begin
      $dumpfile("dump.vcd");
      $dumpvars(1);
    end
endmodule



// full adder
module full_adder(input logic a, b, C_in, 
                  output logic S, C_out); 
 
  always @(a,b,C_in)
    begin
      S <= a^b^C_in; 
      C_out <= a&a |(a^b)&C_in; 
   end  
  
endmodule 

// Ripple carry adder: 
module rc_adder(input  logic[31:0] A, B,
             input  logic C_in,
             output logic[31:0] Y, 
             output logic C_out); 

  logic[30:0] C; 
  
  full_adder dut0(A[0], B[0], C_in, Y[0], C[0]); 
  full_adder dut1(A[1], B[1], C[0], Y[1], C[1]); 
  full_adder dut2(A[2], B[2], C[1], Y[2], C[2]); 
  full_adder dut3(A[3], B[3], C[2], Y[3], C[3]); 
  full_adder dut4(A[4], B[4], C[3], Y[4], C[4]); 
  full_adder dut5(A[5], B[5], C[4], Y[5], C[5]); 
  full_adder dut6(A[6], B[6], C[5], Y[6], C[6]); 
  full_adder dut7(A[7], B[7], C[6], Y[7], C[7]); 
  full_adder dut8(A[8], B[8], C[7], Y[8], C[8]); 
  full_adder dut9(A[9], B[9], C[8], Y[9], C[9]); 
  full_adder dut10(A[10], B[10], C[9], Y[10], C[10]); 
  full_adder dut11(A[11], B[11], C[10], Y[11], C[11]); 
  full_adder dut12(A[12], B[12], C[11], Y[12], C[12]); 
  full_adder dut13(A[13], B[13], C[12], Y[13], C[13]); 
  full_adder dut14(A[14], B[14], C[13], Y[14], C[14]); 
  full_adder dut15(A[15], B[15], C[14], Y[15], C[15]); 
  full_adder dut16(A[16], B[16], C[15], Y[16], C[16]); 
  full_adder dut17(A[17], B[17], C[16], Y[17], C[17]); 
  full_adder dut18(A[18], B[18], C[17], Y[18], C[18]); 
  full_adder dut19(A[19], B[19], C[18], Y[19], C[19]); 
  full_adder dut20(A[20], B[20], C[19], Y[20], C[20]); 
  full_adder dut21(A[21], B[21], C[20], Y[21], C[21]); 
  full_adder dut22(A[22], B[22], C[21], Y[22], C[22]); 
  full_adder dut23(A[23], B[23], C[22], Y[23], C[23]); 
  full_adder dut24(A[24], B[24], C[23], Y[24], C[24]); 
  full_adder dut25(A[25], B[25], C[24], Y[25], C[25]); 
  full_adder dut26(A[26], B[26], C[25], Y[26], C[26]); 
  full_adder dut27(A[27], B[27], C[26], Y[27], C[27]); 
  full_adder dut28(A[28], B[28], C[27], Y[28], C[28]); 
  full_adder dut29(A[29], B[29], C[28], Y[29], C[29]); 
  full_adder dut30(A[30], B[30], C[29], Y[30], C[30]); 
  full_adder dut31(A[31], B[31], C[30], Y[31], C_out); 
endmodule 


// ALU 
module ALU_32bits(input logic[31:0] a, b, 
                  input logic[2:0] F, 
                  output logic[31:0] Y, 
        		  output logic Cout); 
	
  logic Cout_ext, C_in; 
  logic[31:0] S, b_switched; 
  
  rc_adder dut(a, b_switched, F[2], S, Cout_ext); 
  
  assign b_switched = F[2] ? ~b : b; 

  always @(a,b,F,S) 
    begin
    case(F[1:0])
      2'b00: assign Y = a&b;
      2'b01: assign Y = a|b;
      2'b10: assign Y = S;
      2'b11: assign Y = {31'b0, S[31]}; 
    endcase
  end

endmodule 
```

#### Exercise 5.10


(a) <br>

<b>Method 1</b>: <br>  
Overflow and underflow when summing integers expressed as two's complement can only happen when both input numbers $A$ and $B$ are positive, or negative. i.e. when $\overline{A[n-1]\oplus B[n-1]}$. Otherwise, the magnitude of the output number $S$ cannot exceed the magnituded of both $A$ and $B$ and no overflow/underflow is possible. 

The second cue to catching overflow/underflow is if both $A$ and $B$ are positive and the $S[n-1] = 1$. 
The third cue to catching overflow/underflow is if both $A$ and $B$ are negative and the adder's carry-out is 1. 

The final equation should be: 
$A_{n-1}B_{n-1}C_{out} + \bar{A}_{n-1}\bar{B}_{n-1}S_{n-1} $

<b>Method 2</b>: <br>  


The second approach requires to modify the CPA adder directly instead of the ALU. Below is the truth table governing the MSB's full adder. A simpler overflow detection can be calculated with the equation: $C_{out}\oplus C_{in}$   
![alt text](images\P5_10.PNG "Title")


(b)<br> 
The circuit for the second method in (a) is shown below: <br> 

![alt text](images\P5_10_circuit.PNG "Title")


(c) <br> 
For convenience, the overflow output can be forced to 0 when a ALU is requested an operation other than an additional or subtraction. 

HDL for Overflow circuit: <br> 
```
// ALU 
module ALU_32bits(input logic[31:0] a, b, 
                  input logic[2:0] F, 
                  output logic[31:0] Y, 
        		  output logic Cout,
                  output logic overflow); // added overflow output 
	
  logic Cout_ext, C_in; 
  logic[31:0] S, b_switched; 
  
  rc_adder dut(a, b_switched, F[2], S, Cout_ext); 
  
  assign b_switched = F[2] ? ~b : b; 

  
  always @(a,b,F,S, overflow) 
    begin
      case(F[1:0])
        2'b00: assign Y = a&b;
        2'b01: assign Y = a|b;
        2'b10: assign Y = S;
        2'b11: assign Y = {31'b0, S[31]}; 
      endcase
      // method: 1 used to calculate overflow: 
      assign overflow = a[31]&b_switched[31]&Cout_ext | ~(a[31])&(~b_switched[31])&S[31]; 
      
  end
endmodule 

```

#### Exercise 5.11


#### Exercise 5.12

#### Exercise 5.13
 
Terminate the 2 most significant bits, set $B_{i} = A_{i-2}$ and ground $B_0$ and $B_1$
![alt text](images\P5_13.PNG "Title")

#### Exercise 5.14
4-bit left and right rotator applicable to an $n-$bit bus can be built with $n$ 2:1 multiplexers. In the example below, a 7-bit number is used. Setting $F_0$ to 0 results in a 4-bit left rotator. Conversely, setting $F_1$ to 1 results in a 4-bit right rotator. 

![alt text](images\P5_14.PNG "Title")

<b> Verilog module Code</b>

```
module rotator_4bit
  #(parameter BUS_WIDTH = 32)
  (input logic[BUS_WIDTH:0] a, 
   input logic f, 
   output logic[BUS_WIDTH:0] b); 
  
  always@(a,f)
    begin
      if(~f) // left rotation by 4 bits 
        begin
          b[31:4] = a[27:0]; 
          b[3:0] = a[31:28];
        end
      else // right rotation by 4 bits 
        begin
          b[27:0] = a[31:4]; 
          b[31:28] = a[3:0]; 
        end
    end
endmodule 
```

<b> TestBench Code</b>
```
module Q5_14_test(); 
  
  logic[31:0]a,b; 
  logic f; 
  
  rotator_4bit inst(a,f,b);
  defparam inst.BUS_WIDTH = 32; 
  initial 
    begin 
      a = 32'hfff; f = 1; #1; 
      a = 32'hfff; f = 0; #1; 
    end  
    //Optional: save signals to display them later on EDAplayground's EPWave 
    initial 
      begin
      $dumpfile("dump.vcd");
      $dumpvars(1); 
      end
  
endmodule
```

#### Exercise 5.15
![alt text](images\P5_15.PNG "Title")

#### Exercise 5.16

$N$ 2:1 multiplexers can first be used to shift all bites by either an increment of $2^0$ or zero. Then, another row of $N$ 2:1 multiplexers can used to shift the output of the previous row by an increment of $2^1$ or zero. In total, $\log_2N$ rows are sufficient to allows for all combination of shift for increments ranging from 0 to n-1. This is analogous to needing $\log_2N$ bits to express all binary number from $0$ to $n-1$ where the $i^{th}$ bit of the binary number represent a shift of $2^i$. 

#### Exercise 5.17

(a) $B$ = full of zeros, $C$ = $A$, $k$ = <i>shamt</i>.

(b) $B$ = full of $A$'s MSB, $C$ = $A$, $k$ = <i>shamt</i>.

(c) $C$ = full of zeros, $B$ = $A$, $k$ = $N-$<i>shamt</i>.

(d) $B$ = $C$ =$A$ and $k$ = <i>shamt</i>.  

(e) $B$ = $C$ = $A$ and $k$ = $N -$<i>shamt</i>.  

#### Exercise 5.18
Delay for 4x4 multiplier: <br> 
$t_{AND} + 8t_{FA}$ <br> 
Delay for NxN multiplier: <br>
$t_{AND} + (3N-4)t_{FA}$ <br> 

#### Exercise 5.19
Delay for 4x4 divider: <br>
$4(t_{FA}+t_{MUX}) + t_{inv}$ <br>
Delay for NxN divider: <br>
$N(t_{FA}+t_{MUX}) + t_{inv}$ <br>

#### Exercise 5.20

A 2's complement multiplier circuit could easily be designed around 3 stage: Firstly, invert and add 1 to any input that is negative to obtain their magnitude. Secondly, multiply their magnitudes using the unsigned integer multiplier presented in 5.2.6. Lastly, invert and add 1 to the ouput if exactly one of two input was negative. This solution is trivial and easy to implement in a circuit digram. However, there are more efficient approaches. 

The <b>Baugh-Wooley</b> method [1] can be used instead to limit the number of gates. Here's an example for 5-bit number product.  <br>

1. Recall 2's complement are just like unsigned integers where the MSB has a negative weight. This means the product can be carried out, but the negative weight must be taken into account. Hence, the last partial product is negative, and the MSBs are red represent the sign of each partial product. For convenience, signs are extended:  <br>
![alt text](images\P5_20_step2.PNG "Title")

2. The last partial product can added instead of subtracted if its bits are flipped and incremented. Ones are also  added and subtracted to get rid of the left-most extending part of the partial products during the next step: <br>
![alt text](images\P5_20_step3.PNG "Title")

3. Adding ones to the partial product will either turn them positive or increase their magnitude if they were already positive. Either way, the bits to which they were added flip signs and the bits on their left go to zero. 
![alt text](images\P5_20_step4.PNG "Title")
=
4. The remaining bits to subtract can instead be added if they flipped and incremented. The amount of hardware needed is very similar to the unsigned product method.
![alt text](images\P5_20_step5.PNG "Title")

The resulting circuit is shown below: <br>

![alt text](images\P5_20_circuit.PNG "Title")

[1] Refer to slide 30: http://web.mit.edu/6.111/www/f2016/handouts/L08.pdf



#### Exercise 5.21

![alt text](images\P5_21.PNG "Title")
<b>verilog code</b>
```
module signExtender(input logic[3:0] a, 
                    output logic[7:0] b);
  assign b = {{4{a[3]}},a};
endmodule
```    
    


#### Exercise 5.22

![alt text](images\P5_21.PNG "Title")
<b>verilog code</b>
```    
module zeroExtension(input logic[3:0] a, 
                    output logic[7:0] b);
  assign b = {0'b000,a};
endmodule
```

#### Exercise 5.23
![alt text](images\P5_23.PNG "Title")

#### Exercise 5.24
(a) 0 to 4095.999755859375 <br>
(b) -2047.999755859375 to 2047.999755859375<br>
(c) -2048.999755859375 to 2047.999755859375


#### Exercise 5.25
(a) 10001101.10010000 $\implies$ 0x8d90 <br> 
(b) 00101010.01010000 $\implies$ 0x2a50 <br> 
(c) 10010001.00101000 $\implies$ 0x9128 <br> 


#### Exercise 5.26
(a) 111110.100000 $\implies$ 0xfa0 <br> 
(b) 010000.010000 $\implies$ 0x410 <br> 
(c) 101000.000101 $\implies$ 0x502 <br> 

#### Exercise 5.27
(a) −13.5625 = -217 / 16 = 1111111100100111 / $2^4$ = 11110010.01110000  $\implies$ 0xf270 <br> 
(b) 42.3125 = 677 / 16 = 0000001010100101 / $2^4$ = 00101010.01010000  $\implies$ 0x2a50<br>
(c) −17.15625 = -549 / 32 = 1111110111011011 / $2^5$ = 11101110.11011000  $\implies$ 0xeed8 <br> 




#### Exercise 5.28
(a) -30.5  = -61/  2 = 111111000011/ 2 = 100001.100000 $\implies$ 0x860<br>
(b) 16.25  = 65 / 4 = 000001000001 / 4 = 010000.010000 $\implies$ 0x410<br>
(c) -8.078125 = -517 / 64 = 110111111011 / 64 = 110111.111011 $\implies$ 0xdfb<br>




#### Exercise 5.29
half of the problem is already solved in 2.25: 

(a) -13.5625 = 10001101.10010000 = - 1.1011001 x $2^3$ <br>
<b>Sign:</b> 1 <br> 
<b>Exponent: </b>10000010 <br>
<b>Mantissa:</b> 10110010000000000000000 <br>
0xc1590000 <br>


(b) 42.3125 = 00101010.01010000 = 1.010100101 x $2^5$ <br> 
<b>Sign:</b> 0 <br> 
<b>Exponent: </b>10000100 <br>
<b>Mantissa:</b> 01010010100000000000000 <br>
0x42294000 <br>


(c) -17.15625 = 10010001.00101000 = -1.000100101 x $2^4$ <br>
<b>Sign:</b> 1 <br> 
<b>Exponent: </b>10000011 <br>
<b>Mantissa:</b> 00010010100000000000000 <br>
0xc1894000 <br>

#### Exercise 5.30

(a) 111110.100000  = -1.11101 x $2^4$ <br>
<b>Sign:</b> 1 <br> 
<b>Exponent: </b>10000011 <br>
<b>Mantissa:</b> 11101000000000000000000 <br>
0xC1F40000 <br>


(b) 010000.010000 = 1.0000010000 x $2^4$ <br>
<b>Sign:</b> 0 <br> 
<b>Exponent: </b>10000011 <br>
<b>Mantissa:</b> 00000100000000000000000 <br>
0x41820000 <br>


(c) 101000.000101  = 1.000000101x $2^3$ <br>
<b>Sign:</b> 1 <br> 
<b>Exponent: </b>10000010 <br>
<b>Mantissa:</b> 00000010100000000000000 <br>
0xC1014000 <br>


#### Exercise 5.31
#### Exercise 5.32
#### Exercise 5.33


The shift intents to equalize the exponent part of both numbers to permit the summation of both number's mantissa (fraction). We can either: <br> 
1. Drop the exponent of the larger number to match the lower number's exponent, and then left shift the larger number's mantissa (fraction) by the same amount. 
2. Raise the exponent of the smaller number to match the larger number's exponent, and then right shift the number's mantissa (fraction) by the same amount. 

Both approaches are mathematically valid, however the former is undesirable in practice since the mantissa (fraction) of the larger number will most likely overflow when it is shifted left. On the other hand, the mantissa(fraction) of the smaller number might only lose precision when it is right shifted. 


#### Exercise 5.34


(a) C0123456 + 81C564B7 <br>
1 1000000 000100100011010001010110 +  1 0000001 110001010110010010110111 <br>
-1.000100100011010001010110e$\cdot 2^{-63}$ -1.110001010110010010110111$\cdot 2^{-126}$ <br>
-1.000100100011010001010110e$\cdot 2^{-63}$ -0.0$\cdot 2^{-63}$ <br>
= 0xC0123456<br>

(b) 0xD0B10301 + 0xD1B43203 <br>
1 10100001 01100010000001100000001 + 1 10100011 01101000011001000000011 <br>
-1.01100010000001100000001$\cdot2^{34}$ -1.01101000011001000000011 $\cdot2^{36}$  <br>
-0.0101100010000001100000001$\cdot2^{36}$ -1.01101000011001000000011 $\cdot2^{36}$  <br>
-0.0101100010000001100000001$\cdot2^{36}$ -1.01101000011001000000011 $\cdot2^{36}$  <br>
-1.1100000011100101100001101$\cdot2^{36}$ <br> 
= 0xD1E072C3

(c) 0x5EF10324 + 0x5E039020 <br>
0 10111101 11100010000001100100100 + 0 10111100 00000111001000000100000 <br>
1.11100010000001100100100$\cdot2^{62}$ + 1.00000111001000000100000$\cdot2^{61}$ <br>
1.11100010000001100100100$\cdot2^{62}$ + 0.100000111001000000100000$\cdot2^{62}$ <br>
= 1.10110010110010110011010$\cdot2^{62}$ <br> 
= 0x5ED9659A






#### Exercise 5.35
#### Exercise 5.36
#### Exercise 5.37
#### Exercise 5.38
#### Exercise 5.39
#### Exercise 5.40
#### Exercise 5.41
#### Exercise 5.42
#### Exercise 5.43
#### Exercise 5.44
#### Exercise 5.45
#### Exercise 5.46
#### Exercise 5.47
#### Exercise 5.48
#### Exercise 5.49
#### Exercise 5.50
